In [1]:
#pip install easydl


the cSCC dataset is 10x visium technology; the HER2+ dataset ST w/ 100 micrometer spots.

thus, it is better to test with cSCC data

In [2]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *

#in HIST2ST, collections.Iterable is imported via 
# from HIST2ST import * --> from transformer import* 
# --> from easydl import * --> from .pytorch import *
# from .utils import * --> from collections import Iterable.
# the alias Iterable has been deprecated however,
# so you need a workaround: second answer: https://stackoverflow.com/questions/72032032/importerror-cannot-import-name-iterable-from-collections-in-python
import collections.abc
collections.Iterable = collections.abc.Iterable


from HIST2ST import *
from dataset import ViT_HER2ST, ViT_SKIN, ViT_LIVER
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

[easydl] tensorflow not available!


clean memory functions from fastai22p2 notebook 11

In [3]:
import sys
import traceback
import gc

def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''

def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()

clean_mem()

# Data Loading

In [4]:
name=[*[f'A{i}' for i in range(2,7)],*[f'B{i}' for i in range(1,7)],
      *[f'C{i}' for i in range(1,7)],*[f'D{i}' for i in range(1,7)],
      *[f'E{i}' for i in range(1,4)],*[f'F{i}' for i in range(1,4)],*[f'G{i}' for i in range(1,4)]]
patients = ['P2', 'P5', 'P9', 'P10']
reps = ['rep1', 'rep2', 'rep3']
skinname = []
for i in patients:
    for j in reps:
        skinname.append(i+'_ST_'+j)
device='cuda'
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Hist2ST Prediction

### To run the trained model, please select the trained model and replace the value of the variable fold with the number in the name of the selected trained model.

In [5]:
fold=0
data='liver'
prune='Grid' if data=='her2st' else 'NA'
genes=171 #if data=='cscc' else 785

the model was trained for the 785 HVGs in HER2+ breast cancer samples. thus, if we use default models, we will predict genes that will most likely not have anything to do with the liver. therefore, it does not make sense to use default models in the first step of the thesis.

the selection for the genes has already been made before starting to train as only the selected genes will be normalized and put into self.exp_dict of the dataset

In [6]:
#load dataset with the ViT_HER2ST function if data == "her2st"
#fold is the fold for the LOOCV. 
#note that the first section A1 was excluded, so fold 5 will be B1 instead of A6
testset = pk_load(fold,'test',dataset=data,flatten=False,adj=True,ori=True,prune=prune)

['C73A1']
Loading imgs...
Loading metadata...


the genes were not measured in the liver dataset. now, I have to add them to the features, and add 0 for each spot

"For evaluating the gene expression
prediction accuracy, we conducted leave-one-out crossvalidation. Specifically, for each section, we used the left
sections to train our model and predict gene expression
for the section. All predictions were collected to assess
the model performance"

In [7]:
test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)

the next two lines had to be commented out for the model to run on the cSCC dataset

In [8]:
#label=testset.label[testset.names[0]]
#genes=785

params: 
heads = number of attention pooling heads per multi-head attention layer = 16

depth2 = number of stacked multi-head attention layers = 8

internally: 

dim = number of filters of input patch ("we set both input and hidden dimension = 1024") 

dim_head = number of filters analyzed in each of the heads = 64 default

the embedding of patches goes RGB 3 -> "?" -> 1024 -> dim_head * head = 64 * 16 (increased number of filters for each patch now fed into the multi-head attention layer -> divide the 64 * 16 filters up equally, so that each of the 16 heads gets 64 * 16 / 16 = 64 filters (=dim_head) to perform attention on. 

I dont know why they chose dim_head = 64, but the positional embedding n_pos is also 64 embeddings long.

however, this would fit my observations that THE DIMENSION OF THE POS EMBEDDING IS ALWAYS EQUAL TO THE DIMENSION OF THE INPUT TO THE SPECIFIC HEADS OF THE MULTIHEAD ATTENTION MODULES (-> the dimensions of both patch embedding (filter number) fed into each head as well as position embeddings are the same).
for instance, when checking out the transformer implementations from pytorch/timm/[istar](https://github.com/mrfeldmann/istar/blob/master/vision_transformer.py) /sepal you can see that they use the dimension of 768 for the pos embedding as well as for the input dimension for each head.

thus, they might have arbitrarily chosen 64 

__is there a reason for why the dimensions for the embeddings are equal?__


In [19]:
model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes, 
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
    #n_pos = 100,
)

n_pos 64
dim 1024


In [10]:
#model.load_state_dict(torch.load(f'./model/{fold}-Hist2ST.ckpt'))

In [20]:
model.load_state_dict(torch.load(f'./model/{fold}_skin-Hist2ST.ckpt'))

<All keys matched successfully>

internally, the test function runs pred = model(patch, position, adj)[0] the following data from the test_loader

In [12]:
for patch, position, exp, adj, *_, center in test_loader:
    print(patch.shape, position.shape, exp.shape, adj.shape, center.shape)

torch.Size([1, 2378, 3, 112, 112]) torch.Size([1, 2378, 2]) torch.Size([1, 2378, 171]) torch.Size([1, 2378, 2378]) torch.Size([1, 2378, 2])


In [13]:
position

tensor([[[ 50, 102],
         [ 43,   9],
         [ 47,  13],
         ...,
         [ 31,  77],
         [ 58,  42],
         [ 45,  27]]])

then, in the forward pass of the model (HIST2ST), the position tensor is used as "centers", where these are embedded using self.x_embed(centers[:,:,0])

In [16]:
position[:,:,0].shape, position[:,:,0]

(torch.Size([1, 2378]), tensor([[50, 43, 47,  ..., 31, 58, 45]]))

In [15]:
pred, gt = test(model, test_loader,'cpu')

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

patches shape torch.Size([1, 2378, 3, 112, 112])
centers shape torch.Size([1, 2378, 2])
centers max 72
centers min 17


IndexError: index out of range in self

the following cell shows what data is used inside the test function.
I dont get what the difference between center and position is yet though

In [9]:
for patch, position,exp, adj, *_, center in test_loader:
    print(f"patch: {patch.shape}")
    print(f"position: {position.shape} are the x,y centers for the forward pass. ")
    print(f"Expression: {exp.shape}")
    print(f"adj = adjacency matrix {adj.shape}")
    print(f"center: {center.shape}")

patch: torch.Size([1, 2378, 3, 112, 112])
position: torch.Size([1, 2378, 2]) are the x,y centers for the forward pass. 
Expression: torch.Size([1, 2378, 171])
adj = adjacency matrix torch.Size([1, 2378, 2378])
center: torch.Size([1, 2378, 2])


In [10]:
pred, gt


NameError: name 'pred' is not defined

521 spots (?) x 171 genes

In [15]:
pred.obsm["spatial"].shape

(521, 2)

In [16]:
# spatial are the center (ct) locations of the spots
pred.obsm["spatial"]

array([[4581., 8002.],
       [4586., 8809.],
       [4766., 7408.],
       ...,
       [9634., 8815.],
       [9628., 9224.],
       [9623., 9620.]], dtype=float32)

In [17]:
pred.X.shape

(521, 171)

In [18]:
#get_R??

In [19]:
#returns tuple of all array(r values), array(all pvalues)
len(get_R(pred,gt))

2

In [20]:
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmean(R))


# clus,ARI=cluster(pred,label)
# print('ARI:',ARI)


Pearson Correlation: 0.10740173624584474
